# 네이버 뉴스 핵심키워드 추출
* 입력 : 네이버 뉴스 url, 핵심 키워드 갯수
* 방법 : matrix 혹은 그래프 활용 textrank 구현 이용한 문서 요약
* 출력 : 핵심키워드 리스트(갯수 적용), 원문

In [4]:
import requests
from bs4 import BeautifulSoup as bs
import re
from konlpy.tag import Hannanum
import pandas as pd
import numpy as np
import math

In [5]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=277&aid=0004511484'

In [9]:
def newsKeywords(url, count = 10):
    resp = requests.get(url)
    soup = bs(resp.content)
    Text = soup.select('div._article_body_contents')[0].text\
    .replace('// flash 오류를 우회하기 위한 함수 추가','')\
    .replace('function _flash_removeCallback() {}','').strip()
    body = re.sub(r'\[\w{0,10} ?[가-힣]{2,5} ? 기자|\]|\n|\t|\w{0,5} ?기자 ?[a-zA-Z]*@[a-zA-Z]*.*','', Text)
    
     # 토큰화 & 품사 태깅
    hannanum=Hannanum()
    hanTag = hannanum.pos(body)
    
    # 불용어 처리
    stopPos = re.sub("\(.{1,15}\'(I|S|J|X)\'\),? ?","",str(hanTag))
    onlyText = re.sub("\(|,? ?\'\w\'\)","",stopPos)
    stopwords = re.sub(",? ?\'(은|는|이|가|을|를|ㄹ|ㅁ|ㄴ|ㄴ다|고|었다|어|의|부터|하|것|와|과|며|면|으|나|다|지|로|아|수|에|되|서|있|기)\'","",onlyText)
#     stopwords = re.sub(",? ?\'(은|는|이|가|을|를)\'","",onlyText)
    prodText = eval(stopwords)
    voca = list(set(prodText))
    voca_len = len(voca)
    
    # edge 연결
    edge = pd.DataFrame(np.zeros((voca_len, voca_len), dtype = np.float32), columns = voca, index = voca)
    win = 3
        
    for start in range(voca_len-win+1):
        scene = prodText[start:start+win]
        for w1 in scene:
            for w2 in scene:
                if w1 == w2:
                    continue
                else:
                    edge[w1][w2]=1
                    edge[w2][w1]=1
    
    # ranking score 계산
    summ = edge.sum(axis =1 )
    summ = summ[summ !=0] # 연결 없는 노드 제거
    edge = edge[list(summ.index)] 

    cnt = 0
    Max = 50
    d = 0.85
    threshold = 0.0001
    score = np.ones((len(summ)), dtype = np.float32)

    while cnt != Max:
        cnt+=1
        pre_score = np.copy(score)
        weight = []
        for i in range(len(summ)):
            if summ[i] != 0:
                ser = pre_score[i]* edge.iloc[i]/summ[i]
                weight.append(ser)

        weight = pd.DataFrame(weight)
        score = (1-d) + d*weight.sum()
        if np.fabs(pre_score-score).sum() <= threshold:
            break

    keyList = list(score.sort_values(ascending = False)[:count].index)
    return keyList, body

In [10]:
newsKeywords(url,20)

(['글꼴',
  '부리',
  '화면용',
  '디지털',
  '크',
  '매체',
  '설계',
  '새롭',
  '환경',
  '따르',
  '한글',
  '네이버',
  '함께',
  '예정',
  '완성도',
  '게',
  '등',
  '사용자',
  '맞추',
  '한글꼴'],
 '디지털 환경 맞춰 기술력 더해 사용자와 함께 개발…2021년 무료 공개 네이버는 디지털 환경에 맞춰 한글꼴의 원형을 잇는 화면용 \'마루 부리 글꼴\'을 개발한다고 1일 밝혔다. 프로젝트 초기 단계부터 한글 사용자와 함께 한글꼴의 의미와 방향을 고민하고 새로운 화면용 글꼴을 설계해 나간다는 게 큰 뼈대다. 안상수 한글 타이포그래피 디자이너와 일반 한글 사용자가 함께 새로운 화면용 \'마루 부리 글꼴\'을 설계할 예정이다.네이버에 따르면 부리 글꼴은 조선시대 붓으로 다듬어진 궁체 중 해서체를 인쇄용 활자에 맞게 정리한 글꼴로, 글자 줄기에 부리가 없는 민부리 글꼴과 차이가 있다. 부리 글꼴은 서예에 기본을 두고 있어 손글씨와 같이 미세한 필압 표현과 높낮이가 있는 둥근 획, 감정이 담긴 섬세한 미감을 표현한다. 이에 인간적이고 친숙하며 따뜻한 감성이 담겨 있어 오늘날 신문, 잡지, 동화책 등의 인쇄 매체에 주로 쓰이고 있다.1990년대 화면용 한글 글꼴은 해상도의 한계 등으로 인해 저해상도 화면에서도 일그러짐이 적은 민부리 글꼴을 중심으로 개발돼 왔다. 하지만 디지털 화면 출력 기술이 발전함에 따라 완성도 높은 부리 글꼴 개발에 대한 요구가 커지고 있다. 이에 다양한 디지털 매체 환경에서는 완성도 높은 화면용 부리 글꼴의 개발이 반드시 필요하다는 게 네이버의 설명이다.\'마루 부리 글꼴\' 디자인은 크게 확장성, 가독성, 유용성 3가지 기준으로 설계될 예정이다. 그동안 전통적 의미에서 해석되던 부리의 개념을 확장해 새로운 미감과 안정감을 담는다. 또한 스마트폰에서 긴 글을 잘 읽을 수 있도록 화면에 알맞고 눈이 편안한 글꼴 형태와 구조로 가독성을 높일 방침이다. 이를 바